In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
··········


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
import os
os.chdir("drive/FYP")

In [0]:
! pip install hottbox

In [0]:
import numpy as np
import copy
import time
from hottbox.core import Tensor



class LSSTM:
  def __init__(self, C = 10, max_iter = 100):
    
    self.shape = None
    self.order = None
    self.C = C
    self.max_iter = max_iter
    
    self.weight = None
    self.b = 0
    self.iteration = 0
    self.orig_lb = None
    
    self.eta_history = []
    self.b_history = []
    
  def fit(self, images, labels):
    
    self.shape = images[0].shape
    self.order = images[0].order
    self.orig_lb = list(set(labels))
    
    binary_labels = [1 if x == self.orig_lb[0] else -1 for x in labels]
    #set the target class as 1, the rest is -1
    wn = self.init_weight()
    
    for i in range(self.max_iter):
      print(i)
      for n in range(self.order):
        
        eta = self.cal_eta(wn,n)
        self.eta_history.append(eta)
        Xm = self.cal_xm(images, wn, n)
        
        w, b = self.cal_weight(Xm, binary_labels, eta, self.C)
        w = w / np.linalg.norm(w)
        wn[n] = w
        
      self.weight = wn    
      self.b = b
      self.iteration = i
      self.b_history.append(b)
      if self.converged(): break
        
  def predict(self, images):
    w_n = self.weight
    b = self.b
    predict = []
    dist = []
    for im in images:
        temp = im.copy()
        for n, w in enumerate(w_n):
            temp.mode_n_product(np.expand_dims(w, axis=0), mode=n, inplace=True)
        dist.append(temp.data.squeeze() + b)
        predict.append(np.sign(temp.data.squeeze() + b))
    predict = [self.orig_lb[0] if x == 1 else self.orig_lb[1] for x in predict]
    return predict, dist
    
    
  def init_weight(self):
    wn = []
    for dim in self.shape:
      wn.append(np.random.randn(dim))
    
    return(wn)
  
  def cal_eta(self, wn, n):
    eta = 1
    
    for i in range(self.order):
      if i != n:
        eta *= (np.linalg.norm(wn[i])**2)
    
    return eta


  def cal_xm(self, images, wn, n):
    
    Xm = []
    #Q: copy放前面为什么会改变数值
    for im in images:
      xm = im.copy()
      w = wn.copy()
      for i in range(self.order):
        if i != n:
          xm.mode_n_product(np.expand_dims(w[i], axis=0), mode = i, inplace=True)
      x_m = np.expand_dims(xm.data.squeeze(), axis=0)
      Xm.append(x_m)  
    X_m = np.array(Xm).squeeze()
    #print('X_m:')
    #print(X_m.shape)
    
    return(X_m)
  
  def cal_weight(self, Xm, binary_labels, eta, C):
    
    alphas, b = self.optimizer(Xm, binary_labels, eta, C)
    #print(alphas.shape)
    #print(Xm.shape)
    w = np.dot(alphas.transpose(), Xm).squeeze()
    
    return w, b
  
  def optimizer(self, Xm, binary_labels, eta, C):
    M = Xm.shape[0]
    gamma = 2 * C / eta
    y_train = np.expand_dims(np.array(binary_labels), axis = 1)
    
    omega = np.dot(Xm, Xm.transpose())
    
    left_column = np.expand_dims(np.append(np.array([0]), np.ones(M)), axis=1)
    right_block = np.append(np.expand_dims(np.ones(M), axis=0),  omega + (1/gamma) * np.eye(M), axis=0)
    params = np.append( left_column, right_block, axis=1)
    RHS = np.append(np.array([[0]]), y_train, axis=0)
    result = np.dot(np.linalg.inv(params), RHS)
    
    b = result[0][0]
    alphas = result[1:, :]
    
    return alphas, b
    
    
  def converged(self):
    if len(self.b_history) > 5:
      err1 = np.diff(np.array(self.eta_history[-6:]))
      err2 = np.diff(np.array(self.b_history[-6:]))
      
      if np.all(np.abs(err1) < 1e-8) and np.all(np.abs(err2) < 1e-8):
        return True
      
    return False

In [0]:
import pickle
import numpy as np
import os
import matplotlib.pyplot as plt

def load_file(filename):
  with open (filename,'rb') as fo:
    data = pickle.load(fo, encoding = 'latin1')
    label = data['labels']
    images = data['data']
    Label = np.array(label)
    Images = images.reshape(10000,3,32,32).transpose(0,2,3,1).astype('float')
  return Label, Images

def load_cifar10(root):
  label_temp = []
  image_temp = []
  for i in range (1,6):
    trainfile = os.path.join(root, 'data_batch_%d' %(i,))
    l,i = load_file(trainfile)
    label_temp.append(l)
    image_temp.append(i)
  label_train = np.concatenate(label_temp)
  image_train = np.concatenate(image_temp)
  testfile = os.path.join(root, 'test_batch')
  label_test, image_test = load_file(testfile)
  
  return label_train, label_test, image_train, image_test

def showimage(label, image):
  classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
  num_class = len(classes)
  num_sample = 7
  for order, class_name in enumerate(classes):
    index = np.flatnonzero(label == order)
    idxs = np.random.choice(index, num_sample, replace = False, p = None)
    for i, idx in enumerate(idxs):
      plt_idx = i * num_class + order + 1
      plt.subplot(num_sample, num_class, plt_idx)
      plt.imshow(image[idx].astype('uint8'))
      plt.axis('off')
      if i == 0:
        plt.title(class_name)
  plt.show()
  
def class_extract(images, label, c, num):
    idx1 = np.flatnonzero(label == c[0])
    idx1_new = np.random.choice(idx1, num, replace = False, p = None)
    idx2 = np.flatnonzero(label == c[1])
    idx2_new = np.random.choice(idx2, num, replace = False, p = None)
    idx3 = np.flatnonzero(label == c[2])
    idx3_new = np.random.choice(idx3, num, replace = False, p = None)
    num = idx1_new.shape[0] + idx2_new.shape[0] + idx3_new.shape[0]
    idx = np.hstack((idx1_new,idx2_new,idx3_new))
    idxs = np.random.choice(idx, num, replace = False, p = None)
    class_image = images[idxs]
    class_label = label[idxs]
    
    return class_image, class_label


def tensor_train(images):
    tensor_im = []
    for i in range(images.shape[0]):
        tem = np.squeeze(images[i])
        tensor_im.append(Tensor(tem))
    
    return tensor_im

label_train, label_test, image_train, image_test = load_cifar10('.')
train_im, train_lb = class_extract(image_train, label_train, [0,1,2], 2000)
avg_im = np.sum(train_im, axis = 0)/train_im.shape[0]
test_im, test_lb = class_extract(image_test, label_test, [0,1,2], 1000)
train_im = train_im - avg_im
test_im = test_im - avg_im
print(train_im.shape)
print(train_lb.shape)
Image_train = tensor_train(train_im)
Image_test = tensor_train(test_im)
#Label_train = train_lb
#Label_test = test_lb
set(train_lb)

(6000, 32, 32, 3)
(6000,)


{0, 1, 2}

In [0]:
stm = []
for c in range(3):
  print(c)
  print()
  
  Label_train = np.ones(train_lb.shape)
  Label_test = np.ones(test_lb.shape)
  Label_train[train_lb == c] = -1
  Label_test[test_lb == c]= -1
  stm_tmp = LSSTM(C=10, max_iter=20)
  stm_tmp.fit(Image_train, Label_train)
  stm.append(stm_tmp)

0

0
1
2
3
4
5
1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
2

0
1
2
3
4
5


In [0]:
y1, dist1 = stm[0].predict(Image_test)
y2, dist2 = stm[1].predict(Image_test)
y3, dist3 = stm[2].predict(Image_test)

dist = [dist1, dist2, dist3]
dd = np.array(dist)
y_pre=[]
succ = 0
for i in range(3000):
  y = np.argmin(dd.T[i])
  y_pre.append(y)
  if y == test_lb[i]:
    succ += 1
rate = succ/3000
print(rate)

0.665
